# III. 반도체 박막 기본 딥러닝(baseline code)

---
### 1) 데이터 로드 : 전 단계에서 준비한 트레이닝용 데이터를 로드하여 모델 트레이닝에 사용

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning) # 경고 메시지 안보이게 설정

import gc
gc.collect() # garbage collector : 메모리 관리

In [ ]:
import os
import tensorflow.keras as keras

---
전역 변수 중 일부(디렉토리 이름과 파일 이름 등)는 대문자로  
나머지 변수는 소문자로

In [ ]:
# 파일 경로는 단순히 문자열 연결보다는 os.path.join()을 사용하는 것이 좋음 
DATA_DIR = 'data'
TRAIN_DATA_FILE = 'train-splited.csv'
TRAIN_DATA_PATH = os.path.join(DATA_DIR, TRAIN_DATA_FILE)

In [ ]:
import pandas as pd

train_df = pd.read_csv(TRAIN_DATA_PATH)

In [ ]:
#독립변수와 종속변수를 분리합니다.
train_X = train_df.iloc[:, 5:]
train_Y = train_df.iloc[:, 1:5]

In [ ]:
train_X

In [ ]:
train_Y

---
### 2) 기본적인 네트워크 모델을 만들어서 딥러닝 트레이닝

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
model = Sequential()  # model 초기화 
model.add(Dense(units=160, activation='relu', input_dim=226))  # 첫번째 은닉층  # 226개 feature, 160개 뉴런, relu 함수를 활성화 함수로 사용
model.add(Dense(units=160, activation='relu'))   # 두번째 은닉층  
model.add(Dense(units=160, activation='relu'))   # 세번째 은닉층
model.add(Dense(units=4, activation='linear'))   # 출력층 (4개의 output을 도출해내야되기 때문에 units = 4)

In [ ]:
#모델을 컴파일합니다.
model.compile(loss='mae', optimizer='adam', metrics=['mae'])

In [ ]:
model.summary()

In [ ]:
MODEL_DIR = 'model'
os.makedirs(MODEL_DIR, exist_ok=True)

In [ ]:
MODEL_SUMMARY_FILE = "thin_film_base_model.png"
MODEL_SUMMARY_PATH = os.path.join(MODEL_DIR, MODEL_SUMMARY_FILE)

from tensorflow.keras.utils import plot_model

plot_model(model, to_file = MODEL_SUMMARY_PATH, show_shapes=True)

---
#### (가) 체크포인트를 저장하기 : 에포크마타 모델 파일을 저장하기

In [ ]:
# MODEL_DIR = 'model'

from pathlib import Path

CHK_POINT_DIR = os.path.join(MODEL_DIR, 'chk_point')
Path(CHK_POINT_DIR).mkdir(parents=True, exist_ok=True)
# os.makedirs(CHK_POINT_DIR, exist_ok=True)

CHK_POIN_FILE = '{epoch:d}.h5'
CHK_POIN_PATH =  os.path.join(CHK_POINT_DIR, CHK_POIN_FILE)

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(
    CHK_POIN_PATH,
    monitor='val_loss',
    verbose=1,
    save_best_only=False,
    mode='min',
    save_weights_only=False
)

In [ ]:
callbacks = [checkpoint]

In [ ]:
# batch_size 250일 때, NUC에서 epoch 당 약 22초
# 50 epoch : mae = 20.8 (17분)

epoch_num = 5
batch_size = 500 # 4~5분 소요, mea - 35  # 10000 으로 100 epoch 했을 경우 mae : 36 정도

In [ ]:
import time

In [ ]:
#모델을 학습합니다.
start = time.time() # start time

history = model.fit(train_X, train_Y, epochs=epoch_num, batch_size=batch_size, validation_split = 0.05, callbacks = callbacks)

print("time : ", time.time() - start, " sec") # sec

In [ ]:
print(history.history.keys())

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# loss 측정값의 시각화.  

loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1,len(loss)+1)

plt.plot(epochs,loss,label='Training Loss')
plt.plot(epochs,val_loss,label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

In [ ]:
# 저장된 체크포인트 파일들 중 가장 로스 값이 작은 파일을 로드
chk_file = str(epoch_num) + '.h5'
model_file = os.path.join(CHK_POINT_DIR, chk_file)
model_loaded = keras.models.load_model(model_file)

In [ ]:
model_loaded.summary()

In [ ]:
epoch_num = 5
batch_size = 500 # 4~5분 소요, mea - 35 # 10000 으로 100 epoch 했을 경우 mae : 36 정도

In [ ]:
#로드한 모델을 연속해서 학습합니다.
start = time.time() # start time

history_loaded = model_loaded.fit(train_X, train_Y, epochs=epoch_num, batch_size=batch_size, validation_split = 0.05, callbacks = callbacks)

print("time : ", time.time() - start, " sec") # sec

In [ ]:
# loss 측정값의 시각화.  

loss = history_loaded.history['loss']
val_loss = history_loaded.history['val_loss']
epochs = range(1,len(loss)+1)

plt.plot(epochs,loss,label='Training Loss')
plt.plot(epochs,val_loss,label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

---
### 3) 추론

In [ ]:
# 파일 경로는 단순히 문자열 연결보다는 os.path.join()을 사용하는 것이 좋음 
# DATA_DIR = 'data'
TEST_DATA_FILE = 'test-splited.csv'
TEST_DATA_PATH = os.path.join(DATA_DIR, TEST_DATA_FILE)

In [ ]:
import pandas as pd

test_df = pd.read_csv(TEST_DATA_PATH)

In [ ]:
test_df

In [ ]:
#독립변수와 종속변수를 분리합니다.
test_X = test_df.iloc[:, 5:]
test_Y = test_df.iloc[:, 1:5]

In [ ]:
test_X

In [ ]:
test_Y

In [ ]:
#예측값을 생성합니다.
pred_test = model_loaded.predict(test_X)

In [ ]:
pred_test

In [ ]:
from tensorflow.keras.losses import MAE

In [ ]:
mae_mean = MAE(test_Y.to_numpy(), pred_test)

In [ ]:
mae_mean

In [ ]:
mae_mean.shape

In [ ]:
mae_mean[[0]]

In [ ]:
import numpy as np

In [ ]:
np.mean(mae_mean)

In [ ]:
model_loaded.evaluate(test_X, test_Y)

---
### 4) 추론 시각화

In [ ]:
def autolabel(ax, rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{:.1f}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

In [ ]:
def compare_graph(i): # 테스트 데이터에서 i 번째 데이터의 라벨값과 예측값 비교 그래프
    layers = ['layer_1', 'layer_2', 'layer_3', 'layer_4']
    predicts = pred_test[i]
    testY = test_Y.iloc[i:i + 1,:].values[0]

    x = np.arange(len(layers))  # the label locations
    width = 0.35  # the width of the bars

    fig, ax = plt.subplots()
    rects1 = ax.bar(x - width/2, predicts, width, label='predicted value')
    rects2 = ax.bar(x + width/2, testY, width, label='label')

    # Add some text for labels, title and custom x-axis tick labels, etc.
    ax.set_ylabel('nano meter')
    ax.set_title('thickness')
    ax.set_xticks(x)
    ax.set_xticklabels(layers)
    ax.legend()
    
    autolabel(ax, rects1)
    autolabel(ax, rects2)

    fig.tight_layout()

    plt.show()

In [ ]:
compare_graph(1000) 